In [1]:
import os
import yaml
from getpass import getpass

from navigator_helpers import NL2PythonTaskSuite, GretelLLMSuite
from navigator_helpers.tasks.text_to_code.utils import display_nl2code_sample
from navigator_helpers.tasks.prompt_templates.template_suite import load_prompt_template_suite

In [2]:
gretel_api_key = os.environ.get("GRETEL_PROD_API_KEY") or getpass("Enter your Gretel API key: ")

llm_config = yaml.safe_load(f"""\
- model_name: gretelai-gpt-llama3-1-8b
  litellm_params:
    model: gretelai/gpt-llama3-1-8b
    api_key: {gretel_api_key}
    api_base: https://api.gretel.ai
  tags:
  - open_license
  - nl
  - code
  - judge
"""
)

llm_suite = GretelLLMSuite(llm_config=llm_config)

2024-09-23 17:59:42.191 - INFO - 🦜 Initializing LLM suite
2024-09-23 17:59:42.193 - INFO - 📖 Natural language LLM: gretelai-gpt-llama3-1-8b
2024-09-23 17:59:42.193 - INFO - 💻 Code LLM: gretelai-gpt-llama3-1-8b
2024-09-23 17:59:42.194 - INFO - ⚖️ Judge LLM: gretelai-gpt-llama3-1-8b


In [3]:
tasks = NL2PythonTaskSuite(llm_suite)

In [4]:
record = tasks.create_record(
    domain="Physics", 
    topic="Particle Physics", 
    complexity="Novice: Basic syntax, variables, and data types", 
    llm_as_a_judge=False, 
    syntax_validation=False
)

In [5]:
display_nl2code_sample(record)

                                                  Contextual Tags                                                  
┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Domain  ┃ Topic            ┃ Complexity                              ┃ Suggested Packages                       ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Physics │ Particle Physics │ Novice: Basic syntax, variables, and    │ astropy, scikit-learn, pandas, scipy,    │
│         │                  │ data types                              │ numpy, matplotlib, sympy                 │
└─────────┴──────────────────┴─────────────────────────────────────────┴──────────────────────────────────────────┘

╭───────────────────────────────────────── Natural Language Description ──────────────────────────────────────────╮
│ The program calculates the momentum of a particle given its mass and velocity. It asks the user for the mass of │
│ the particle and its velocity, then uses the formula p = m * v to compute the momentum, where p is the          │
│ momentum, m is the mass, and v is the velocity.                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── Code ──────────────────────────────────────────────────────╮
│ import numpy as np                                                                                              │
│                                                                                                                 │
│ def calculate_momentum():                                                                                       │
│     mass = float(input("Enter the mass of the particle in kg: "))                                               │
│     velocity = float(input("Enter the velocity of the particle in m/s: "))                                      │
│     momentum = mass * velocity                                                                                  │
│     print(f"The momentum of the particle is {momentum} kg*m/s")                                                 │
│                                                                                                                 │
│ calculate_momentum()                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [6]:
tags = tasks.generate_contextual_tags(
    num_domains=3,
    num_topics_per_domain=5,
    num_complexity_levels=4,
)

2024-09-23 17:59:46.846 - INFO - 🏷️ Generating domains
2024-09-23 17:59:47.971 - INFO - 🏷️ Generating topics for each domain
2024-09-23 17:59:51.621 - INFO - 🏷️ Generating levels of Python complexity


In [7]:
tags

ContextualTags({
    "domain_and_topics": {
        "Data Science and Analytics": [
            "Predictive Modeling Techniques",
            "Machine Learning Algorithms",
            "Data Visualization Tools",
            "Statistical Data Analysis",
            "Business Intelligence Solutions"
        ],
        "Scientific Research Institutions": [
            "Lab Information Management Systems",
            "Research Data Management Tools",
            "Scientific Computing Platforms",
            "Data Analysis Software Suites",
            "Laboratory Information Systems"
        ],
        "Financial Trading Platforms": [
            "Algorithmic Trading Systems",
            "Cryptocurrency Market Analysis",
            "Portfolio Management Tools",
            "Risk Management Strategies",
            "Trading Platform Development"
        ]
    },
    "complexity_levels": [
        "Beginner: Basic syntax and data types",
        "Intermediate: Functions and control struc

In [8]:
prompt_templates = load_prompt_template_suite("python")

prompt_templates

PromptTemplateSuite(
    domains: ('num_domains',)
    topics_from_domains: ('num_topics', 'domain')
    complexity: ('num_levels',)
    python_natural_language: ('nl_type_description', 'nl_type', 'complexity', 'nl_type', 'domain', 'topic')
    python_code_generation: ('python_natural_language', 'complexity', 'domain', 'topic', 'suggested_packages')
    python_suggested_packages: ('domain', 'max_dependencies', 'domain')
)

In [9]:
print(prompt_templates.domains(num_domains=3))

Create a list of 3 unique industries where you expect to find software engineers who code in Python. 

### Instructions:
    * Do not use abbreviations.
    * Keep each industry name to 1-5 words, preferring concise names.
    * List the industries in a valid JSON array.

